# Accessing Backend Objects from PlotCollection

A common question when working with `PlotCollection` is how to access the underlying
plotting objects (like matplotlib axes) for further customization. This guide shows
you how to do exactly that.

In [ ]:
import arviz_plots as azp
from arviz_base import load_arviz_data

azp.style.use("arviz-variat")

## The `viz` DataTree Structure

Every `PlotCollection` stores its visual elements in the `viz` attribute, which is a
DataTree containing:

- `"chart"`: The actual plotting elements (axes in matplotlib, figures in bokeh/plotly)
- `"plot"`: Alias for chart in some contexts
- `"row"` / `"col"`: Row/column label elements (when using faceting)

Let's create a simple plot and explore its structure:

In [ ]:
# Load example data and create a plot
data = load_arviz_data("centered_eight")
pc = azp.plot_dist(data, var_names=["mu", "tau"], backend="matplotlib")
pc.show()

In [ ]:
# Explore the viz structure
print("Available keys in viz:")
print(list(pc.viz.keys()))

## Getting a Specific Variable's Axis

Use the `get_viz()` method to retrieve the plotting element for a specific variable:

In [ ]:
# Create a fresh plot
pc = azp.plot_dist(data, var_names=["mu", "tau"], backend="matplotlib")

# Get the matplotlib axis for the "mu" variable
ax_mu = pc.get_viz("chart", "mu")

# Now you can use any matplotlib method!
ax_mu.set_xlim(-5, 12)
ax_mu.axvline(0, color='red', linestyle='--', linewidth=2, label='zero reference')
ax_mu.legend()

pc.show()

## Getting the Figure Object

To access the figure for operations like saving:

In [ ]:
pc = azp.plot_dist(data, var_names=["mu", "tau"], backend="matplotlib")

# Get the matplotlib Figure object
fig = pc.viz["chart"].item()

# You can now use figure methods
print(f"Figure size: {fig.get_size_inches()}")

# Save with custom settings (uncomment to save)
# fig.savefig("my_posterior.png", dpi=300, bbox_inches='tight')

pc.show()

## Iterating Over All Axes

When you need to apply changes to all variables:

In [ ]:
pc = azp.plot_dist(data, var_names=["mu", "tau"], backend="matplotlib")

# Get all variable names
var_names = pc.viz["chart"].coords["__variable__"].values
print(f"Variables: {var_names}")

# Apply customizations to each axis
for var_name in var_names:
    ax = pc.get_viz("chart", var_name)
    ax.set_title(f"Posterior of {var_name}", fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)

pc.show()

## Backend-Agnostic Design

The same API works regardless of which backend you're using! The `get_viz()` method
returns the native object for your chosen backend:

| Backend | `get_viz("chart", var)` returns |
|---------|--------------------------------|
| matplotlib | `matplotlib.axes.Axes` |
| bokeh | `bokeh.plotting.Figure` |
| plotly | `plotly.graph_objects.Figure` |

This means you can write flexible code that works across backends, or access
backend-specific features when needed.

## Complete Example

Here's a full example combining these techniques:

In [ ]:
# Create the plot
pc = azp.plot_dist(
    data,
    var_names=["mu", "tau"],
    backend="matplotlib"
)

# Customize individual axes
ax_mu = pc.get_viz("chart", "mu")
ax_mu.axvline(0, color='red', linestyle='--', alpha=0.7, label='μ=0')
ax_mu.set_xlabel("Value", fontsize=10)
ax_mu.legend()

ax_tau = pc.get_viz("chart", "tau")
ax_tau.axvline(1, color='blue', linestyle=':', alpha=0.7, label='τ=1')
ax_tau.legend()

# Access the figure
fig = pc.viz["chart"].item()
fig.suptitle("Customized Posterior Distributions", fontsize=14, y=1.02)
fig.tight_layout()

pc.show()